In [ ]:
!pip install transformers datasets comet-ml

In [ ]:
import os

os.environ['COMET_API_KEY'] = "<COMMET API KEY>"

import comet_ml

# Commet Init
comet_ml.init(project_name="DNA_Deberta_Small", api_key= "<COMMET API KEY>")

COMET INFO: Comet API key is valid
COMET WARNING: running in Google Colab, but can't find mounted drive. Using HOME instead
COMET WARNING: if drive is mounted, set COMET_CONFIG to save config there
COMET INFO: Comet API key saved in /root/.comet.config


In [ ]:
import torch 

torch.cuda.get_device_name(0)

'Tesla V100-SXM2-16GB'

In [ ]:
from datasets import load_dataset
datasets = load_dataset("simecek/Human_DNA_v0_DNABert6tokenized")
datasets.set_format("torch")
datasets

Downloading:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

Using custom data configuration simecek--Human_DNA_v0_DNABert6tokenized-0a0afa978bceef3e


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/simecek___parquet/simecek--Human_DNA_v0_DNABert6tokenized-0a0afa978bceef3e/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 95417
    })
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 858737
    })
})

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification, DataCollatorForLanguageModeling

tokenizer = AutoTokenizer.from_pretrained("armheb/DNA_bert_6")

tokenizer.decode([0,1,2,3,4,5])

Downloading:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

'[PAD] [UNK] [CLS] [SEP] [MASK] AAAAAA'

## 1) Training

In [ ]:
from transformers import DebertaConfig, DebertaForMaskedLM, TrainingArguments, Trainer

model_config = DebertaConfig(vocab_size=len(tokenizer.vocab), max_position_embeddings=512, num_hidden_layers=6)
model_config

DebertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": null,
  "position_biased_input": true,
  "relative_attention": false,
  "transformers_version": "4.19.2",
  "type_vocab_size": 0,
  "vocab_size": 4101
}

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.2
)

model = DebertaForMaskedLM(config=model_config)
sum(p.numel() for p in model.parameters()) // 10**6

46

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
training_args = TrainingArguments(
    output_dir='./model',          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,      
    num_train_epochs=10,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=32, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=2,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=32,  # evaluation batch size
    logging_steps=10000,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=10000,
    fp16=True,
    load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    save_total_limit=3,           # whether you don't have much space so you let only 5 model weights saved in the disk
    push_to_hub=True,
    hub_model_id="DNADebertaSmall2",
    hub_strategy="every_save"
)

using `logging_steps` to initialize `eval_steps` to 10000
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=datasets['train'],
    eval_dataset=datasets['test'],
)

Cloning https://huggingface.co/simecek/DNADebertaSmall2 into local empty directory.
Using amp half precision backend


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 858737
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 2
  Total optimization steps = 134180
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `

Step,Training Loss,Validation Loss
10000,7.288600,7.256587
20000,7.217600,7.026752
30000,6.880100,6.716784
40000,6.641800,6.531048
50000,6.480500,6.381609
60000,6.373500,6.305342
70000,6.307800,6.247981


***** Running Evaluation *****
  Num examples = 95417
  Batch size = 32
Saving model checkpoint to ./model/checkpoint-10000
Configuration saved in ./model/checkpoint-10000/config.json
Model weights saved in ./model/checkpoint-10000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 95417
  Batch size = 32
Saving model checkpoint to ./model/checkpoint-20000
Configuration saved in ./model/checkpoint-20000/config.json
Model weights saved in ./model/checkpoint-20000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 95417
  Batch size = 32
Saving model checkpoint to ./model/checkpoint-30000
Configuration saved in ./model/checkpoint-30000/config.json
Model weights saved in ./model/checkpoint-30000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 95417
  Batch size = 32
Saving model checkpoint to ./model/checkpoint-40000
Configuration saved in ./model/checkpoint-40000/config.json
Model weights saved in ./model/checkpoint-40000/pytorch_model.bin


Step,Training Loss,Validation Loss
10000,7.288600,7.256587
20000,7.217600,7.026752
30000,6.880100,6.716784
40000,6.641800,6.531048
50000,6.480500,6.381609
60000,6.373500,6.305342
70000,6.307800,6.247981


***** Running Evaluation *****
  Num examples = 95417
  Batch size = 32
